In [2]:
import sys
from opentrons import protocol_api, execute, simulate
from opentrons.types import Location, Point
import pandas as pd
import numpy as np
import urllib.request
import time
import ast  # safer than eval
#imports take roughly 1 minute

# Before running the program, please setup a server connection for a shared .csv folder. 

Run this in the terminal of the central computer: 

```
cd: "your protocol folder path"
python -m http.server 8000
```

This will open a sever to allow opentron to acess the file from: 

csv_path = "http://your pc IP HERE:8000/volumes.csv"

In [22]:
# example using the Sun Lab computer
path = "http://192.168.0.117:8000/BO8.csv"
df = pd.read_csv(path)
# Get first and last 8 rows
df_first = df.head(0)
df_last = df.tail(8)

# Combine them into one DataFrame (optional)
df_combined = pd.concat([df_first, df_last], ignore_index=True)
df_combined # Display volumes to verify

,id,well,volC,volM,volY,volK
0,65,A8,71.0,0.0,71.0,58.0
1,66,B8,0.0,100.0,0.0,100.0
2,67,C8,0.0,83.6,0.0,116.4
3,68,D8,66.7,66.7,66.7,0.0
4,69,E8,NaN,NaN,NaN,NaN
5,70,F8,0.0,43.0,78.5,78.5
6,71,G8,35.7,0.0,128.6,35.7
7,72,H8,89.4,21.3,89.4,0.0


# Define functions

In [4]:
# metadata
metadata = {
    "protocolName": "IMOD Protocol",
    "author": "Clara Tamura, Austin Martin",
    "description": "Mara OT-2 Protocol for IMOD project",
    "apiLevel": "2.19",
}

class OpentronMara:
    def __init__(self, protocol):
        self.protocol = protocol
        self.protocol._commands.clear()

        # Load tipracks
        self.tiprack_300 = protocol.load_labware('opentrons_96_tiprack_300ul', '2')

        # Load Pipettes
        self.pip300 = protocol.load_instrument('p300_single_gen2', mount='right', tip_racks=[self.tiprack_300])

        # Track used tips manually
        self.tip_log = {
            self.pip300: {'tips': self.tiprack_300.wells(), 'next': 0},
        }

        # Set pipette buffers
        self.pip300_buffer = 20

        self.pip300.flow_rate.aspirate = 50
        self.pip300.flow_rate.dispense = 50
        self.pip300.flow_rate.blow_out = 100

        # Load plates
        self.plate_96 = protocol.load_labware('opentrons_96_wellplate_200ul_pcr_full_skirt', '1')
        self.plate_96.set_offset(x=0, y=0, z=55)  # Z lowered by 1 mm
    
        self.plate_6 = protocol.load_labware('corning_6_wellplate_16.8ml_flat','3')
        self.plate_6.set_offset(x=0, y=0, z=10)
        self.source_wells = ['A1', 'A2', 'B1', 'B2']

        # Home the pipettes
        self.pip300.home()

        print("Initialization complete.")

    def pick_up_tip(self, pipette):
        """Pick up the next available tip for the given pipette, with rack limit check."""
        next_tip_index = self.tip_log[pipette]['next']
        tips = self.tip_log[pipette]['tips']
        
        if next_tip_index >= len(tips):
            raise RuntimeError(f"❌ No more tips available for {pipette.name}. Tip rack is empty!")

        tip = tips[next_tip_index]
        pipette.pick_up_tip(tip)
        self.tip_log[pipette]['next'] += 1

    def reset_tips(self, pipette=None):
        if pipette:
            self.tip_log[pipette]['next'] = 0
        else:
            for pip in self.tip_log:
                self.tip_log[pip]['next'] = 0
        
    def distribute_colors(self, df_combined: pd.DataFrame, columns):
        self.columns = columns
        volumes_df = df_combined[self.columns].round(1)
        num_samples, num_solutions = volumes_df.shape
        well_names = df_combined['well']
        print(well_names)
        destination = [self.plate_96.wells_by_name()[well] for well in well_names]

        
        for i in range(num_solutions):
            self.pip300.distribute(
                volumes_df[self.columns[i]].to_list(),
                self.plate_6.wells(self.source_wells[i])[0],
                destination,
                disposal_vol=0,
                blow_out=True,
                blowout_location='source well',
                new_tip='once',
                trash=False
            )
        self.reset_tips(self.pip300) # Reset the tip log for the pipette after use
        #self.print_cmd()
        
        
    def print_cmd(self):
        print("Printing Opentron Commands:")
        for cmd in self.protocol.commands():
            print(cmd)
    
    def clear_protocol(self):
        self.protocol._commands.clear()




# Initialize the OT-2
This step will take 1.5 minutes

In [5]:
protocol = execute.get_protocol_api('2.19') 
#protocol = simulate.get_protocol_api('2.19')  # Use simulate for testing
robot = OpentronMara(protocol)

/data/robot_settings.json not found. Loading defaults
Failed to initialize character device, will not be able to control gpios (lights, button, smoothiekill, smoothie reset). Only one connection can be made to the gpios at a time. If you need to control gpios, first stop the robot server with systemctl stop opentrons-robot-server. Until you restart the server with systemctl start opentrons-robot-server, you will be unable to control the robot using the Opentrons app.
/data/deck_calibration.json not found. Loading defaults


Initialization complete.


# Execute pipetting command

In [23]:
robot.distribute_colors(df_combined, ['volC', 'volM', 'volY', 'volK'])

robot.protocol._commands.clear() # Clear the command queue
robot.tiprack_300.reset()  # Reset the tip racks to ensure a clean state

0    A8
1    B8
2    C8
3    D8
4    E8
5    F8
6    G8
7    H8
Name: well, dtype: object


Polling exception
Traceback (most recent call last):
  File "/usr/lib/python3.10/site-packages/opentrons/hardware_control/poller.py", line 98, in _poll_once
  File "/usr/lib/python3.10/site-packages/opentrons/hardware_control/modules/heater_shaker.py", line 403, in read
  File "/usr/lib/python3.10/site-packages/opentrons/hardware_control/modules/heater_shaker.py", line 414, in read_rpm
  File "/usr/lib/python3.10/site-packages/opentrons/drivers/heater_shaker/driver.py", line 145, in get_rpm
  File "/usr/lib/python3.10/site-packages/opentrons/drivers/asyncio/communication/serial_connection.py", line 435, in send_command
  File "/usr/lib/python3.10/site-packages/opentrons/drivers/asyncio/communication/serial_connection.py", line 459, in send_data
  File "/usr/lib/python3.10/site-packages/opentrons/drivers/asyncio/communication/serial_connection.py", line 513, in _send_data
opentrons.drivers.asyncio.communication.errors.NoResponse: /dev/ot_module_heatershaker0: No response to 'M123 
'
Pol

Polling exception
Traceback (most recent call last):
  File "/usr/lib/python3.10/site-packages/opentrons/hardware_control/poller.py", line 98, in _poll_once
  File "/usr/lib/python3.10/site-packages/opentrons/hardware_control/modules/heater_shaker.py", line 402, in read
  File "/usr/lib/python3.10/site-packages/opentrons/hardware_control/modules/heater_shaker.py", line 411, in read_temperature
  File "/usr/lib/python3.10/site-packages/opentrons/drivers/heater_shaker/driver.py", line 124, in get_temperature
  File "/usr/lib/python3.10/site-packages/opentrons/drivers/asyncio/communication/serial_connection.py", line 435, in send_command
  File "/usr/lib/python3.10/site-packages/opentrons/drivers/asyncio/communication/serial_connection.py", line 459, in send_data
  File "/usr/lib/python3.10/site-packages/opentrons/drivers/asyncio/communication/serial_connection.py", line 513, in _send_data
opentrons.drivers.asyncio.communication.errors.NoResponse: /dev/ot_module_heatershaker0: No response

# If user interrupts the program, reset the robot

In [7]:
#robot.protocol.home()    # Home the robot
robot.pip300.drop_tip()  # Ensure the pipette is not holding a tip before starting

robot.protocol._commands.clear() # Clear the command queue
robot.tiprack_300.reset()  # Reset the tip racks to ensure a clean state

Polling exception
Traceback (most recent call last):
  File "/usr/lib/python3.10/site-packages/opentrons/hardware_control/poller.py", line 98, in _poll_once
  File "/usr/lib/python3.10/site-packages/opentrons/hardware_control/modules/heater_shaker.py", line 402, in read
  File "/usr/lib/python3.10/site-packages/opentrons/hardware_control/modules/heater_shaker.py", line 411, in read_temperature
  File "/usr/lib/python3.10/site-packages/opentrons/drivers/heater_shaker/driver.py", line 124, in get_temperature
  File "/usr/lib/python3.10/site-packages/opentrons/drivers/asyncio/communication/serial_connection.py", line 435, in send_command
  File "/usr/lib/python3.10/site-packages/opentrons/drivers/asyncio/communication/serial_connection.py", line 459, in send_data
  File "/usr/lib/python3.10/site-packages/opentrons/drivers/asyncio/communication/serial_connection.py", line 513, in _send_data
opentrons.drivers.asyncio.communication.errors.NoResponse: /dev/ot_module_heatershaker0: No response